### Import Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


### Define Configs

In [2]:
tickers = {
    'MSFT': {'ipo': '1986-03-13'},
    'AAPL': {'ipo': '1980-12-12'},
    'NVDA': {'ipo': '1999-01-22'},
    'AMZN': {'ipo': '1997-05-15'},
    'GOOG': {'ipo': '2004-08-19'},
    'META': {'ipo': '2012-05-18'},
    'TSLA': {'ipo': '2010-06-29'}
}

initial_capital = 1_000_000
commission = 0.001  # 0.10%
slippage = 0.0002  # 0.02%
min_shares = 10
risk_free_rate = 0.02
trading_days = 252  # Annual trading days
start_date = "1981-01-01"
end_date = "2023-12-31"


### Fetching data

In [3]:
def fetch_data(tickers):
    data = {}
    for ticker, details in tickers.items():
        ipo_date = details['ipo']
        print(f"Fetching data for {ticker} (IPO: {ipo_date})...")

        df = yf.download(ticker, start=ipo_date, end=end_date, auto_adjust=True)

        if df.empty:
            print(f"Warning: No data for {ticker}.")
            continue

        df['Ticker'] = ticker
        data[ticker] = df
    return data

data = fetch_data(tickers)


Fetching data for MSFT (IPO: 1986-03-13)...


[*********************100%***********************]  1 of 1 completed


Fetching data for AAPL (IPO: 1980-12-12)...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetching data for NVDA (IPO: 1999-01-22)...



[*********************100%***********************]  1 of 1 completed


Fetching data for AMZN (IPO: 1997-05-15)...
Fetching data for GOOG (IPO: 2004-08-19)...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetching data for META (IPO: 2012-05-18)...
Fetching data for TSLA (IPO: 2010-06-29)...


### Compute RSI

In [4]:
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window, min_periods=1).mean()
    avg_loss = loss.rolling(window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

for ticker in data:
    df = data[ticker]
    df['RSI'] = compute_rsi(df['Close'])


In [7]:
signals = {}
for ticker in data:
    df = data[ticker]
    df['Signal'] = np.where(df['RSI'] < 25, 'buy', np.where(df['RSI'] > 75, 'sell', None))
    df['Position'] = None
    signals[ticker] = df[['Signal', 'Position']]


In [8]:
all_dates = pd.date_range(start=start_date, end=end_date)
signals_df = pd.DataFrame(index=all_dates)

for ticker in tickers:
    if ticker in signals:
        signals_df[ticker] = signals[ticker]['Signal']

positions_df = signals_df.copy()
positions_df[:] = None  # Start with no positions

for ticker in tickers:
    if ticker in signals:
        prev_position = None
        for date in signals_df.index:
            current_signal = signals_df.at[date, ticker] if date in signals_df.index else None

            if current_signal == 'buy':
                prev_position = 1  # Long position
            elif current_signal == 'sell':
                prev_position = 0  # Exit position
            else:
                prev_position = prev_position  # Maintain previous position

            positions_df.at[date, ticker] = prev_position


In [9]:
signals

{'MSFT': Price      Signal Position
 Ticker                    
 Date                      
 1986-03-13   None     None
 1986-03-14   sell     None
 1986-03-17   sell     None
 1986-03-18   None     None
 1986-03-19   None     None
 ...           ...      ...
 2023-12-22   None     None
 2023-12-26   None     None
 2023-12-27   None     None
 2023-12-28   None     None
 2023-12-29   None     None
 
 [9527 rows x 2 columns],
 'AAPL': Price      Signal Position
 Ticker                    
 Date                      
 1980-12-12   None     None
 1980-12-15    buy     None
 1980-12-16    buy     None
 1980-12-17    buy     None
 1980-12-18   None     None
 ...           ...      ...
 2023-12-22   None     None
 2023-12-26   None     None
 2023-12-27   None     None
 2023-12-28   None     None
 2023-12-29   None     None
 
 [10853 rows x 2 columns],
 'NVDA': Price      Signal Position
 Ticker                    
 Date                      
 1999-01-22   None     None
 1999-01-25   sell     

In [10]:
positions_df['Position']

KeyError: 'Position'

In [ ]:
# TODO: Backtest